### Introduction to Datahub
Energy Exemplar Datahub aims to be a robust storage solution that supports access control and versioning capability and can be used for any customer data. A robust change tracking mechanism means only deltas are synced resulting in less bandwidth used compared to uploading whole files. 
Datahub allows for individual files to be uploaded, downloaded, searched for, deleted and un-deleted or entire directory structures to be syncronized between desktop and cloud. 

### Covered in this Example
1. Python libraries, environment variables and SDK setup
2. Setting SDK/CLI Environment
3. Authentication (SSO)
4. Authentication (Service Principal)
5. Configure Datahub Sync directory
6. Unmapping Datahub Sync directory
7. Syncing a mapped directory
8. Uploading files
9. Downloading files
10. Using a manifest file to download files
11. Searching for files
12. Reverting changes to files
13. Deleting files
14. UnDelete files


In [ ]:
import os
import sys
from eecloud.cloudsdk import CloudSDK, SDKBase
from eecloud.models import *

#### Setup Variables

In [ ]:
# For client credential flow
#------------------------------
tenant_id = 'TENANT_ID'
client_id = 'CLIENT_ID'
client_secret = 'SECRET'
#------------------------------
repo_name = "JupyterSamples"
jupyter_user = os.environ.get("JUPYTERHUB_USER")
cli_path: str = "/usr/local/bin/plexos-cloud"
environment: str = "PreProd"
local_map_folder: str = "./MappedFolder/"
datahub_mapped_folder: str = f"{repo_name}/{jupyter_user}_sync"

working_folder: str = "./WorkingFolder/"
datahub_working_folder: str = f"{repo_name}/{jupyter_user}_work"

#### Set the CLI client from the default installation path (Windows Only)

In [ ]:
pxc = CloudSDK()

#### Set the CLI client from a custom location (Windows & Linux)

In [ ]:
pxc = CloudSDK(cli_path)

#### Set Environment

In [ ]:
try:
    env_response: list[CommandResponse[Contracts_EnvironmentResponse]] = pxc.environment.set_user_environment(environment)
    env_data: Contracts_EnvironmentResponse = SDKBase.get_response_data(env_response)

    print(f"Selected Environment: {env_data.Environment}")
    
except Exception as ex:
    print(ex)

#### Login with SSO (Windows only)

In [ ]:
#try:
#    login_response: list[CommandResponse[Contracts_LoginResponse]] = pxc.auth.login()
#    login_data: Contracts_LoginResponse = SDKBase.get_response_data(login_response)
#    print(f"Tenant: {login_data.TenantName}, User: {login_data.UserName}")
#except Exception as ex:
#    print(ex)

### Login with Client Credentials

In [ ]:
try:
    login_response: list[CommandResponse[Contracts_LoginResponse]] = pxc.auth.login_client_credentials(use_client_credentials=True, client_id=client_id, client_secret=client_secret, tenant_id=tenant_id)
    login_data: Contracts_LoginResponse = SDKBase.get_response_data(login_response)

    print(f"Tenant: {login_data.TenantName}, User: {login_data.UserName}")
    
except Exception as ex:
    print(ex)

#### Map a Local folder to a remote Datahub folder
Note: The folder must be initially empty or files may get overwritten from the server at first sync

In [ ]:
try:
    map_response: list[CommandResponse[Contracts_DatahubMapResponse]] = pxc.datahub.map_folder(local_map_folder, datahub_mapped_folder, print_message=True)
    map_data: Contracts_DatahubMapResponse = SDKBase.get_response_data(map_response)

    if map_data is not None:
        print(f"Map success: {map_data.Success}, Local Path: {map_data.LocalPath}, Remote Path: {map_data.RemotePath}, Patterns: {map_data.Patterns}")
    else:
        print(f"Mapping already exists!")
        
except Exception as ex:
    print(ex)

#### Unmap a folder

In [ ]:
try:
    unmap_response: list[CommandResponse[Contracts_DatahubUnDeleteResponse]] = pxc.datahub.unmap_folder(local_map_folder, print_message=True)
    unmap_data: Contracts_DatahubUnDeleteResponse = SDKBase.get_response_data(unmap_response)

    if unmap_data is not None:
        print(f"Success: {unmap_data.Success}")
    else:
        print("There is nothing to unmap at for this path!")

except Exception as ex:
    print(ex)

#### Sync a specific mapped folder
You can setup extra options:
- Verify downloads: 
    - Build the signature of the downloaded file and compares it to the server to check for data integrity (takes longer).
- Replace local files on conflicts:
    - If the local files are out of sync with the server and there is more that one version of difference, then this option will force the server version and overwrite your local files. 

```
In the case of a conflict and the "replace_local_files_on_conflict is set to False", 
then a ".conflict" file is downloaded locally and your local file is left untouched but not synched to the server.
You will have to use a compare tool locally (like Windiff or Winmerge), 
fix the differences in your file, then use the **Upload command** to send your file to the server, then run a Sync again. 
```

In [ ]:
try:
    sync_response: list[CommandResponse[Contracts_DatahubCommandResponse]] = pxc.datahub.sync(local_path_to_sync=local_map_folder, print_message=True)
    sync_data: Contracts_DatahubCommandResponse = SDKBase.get_response_data(sync_response)

    if sync_data is not None:
        print(f"Success:{sync_data.DatahubCommandStatus.value}")

        for result in sync_data.DatahubResourceResults:
            print(f"Local Path: {result.LocalFilePath}, Datahub Path: {result.RelativeFilePath}, Failure Reason: {result.FailureReason}")
    else:
        print("No changes were found")

except Exception as ex:
    print(ex)

#### Upload (Independent of mappings and sync)
- In the case Datahub already contains the file being uploaded, it will attempt to perform an update
    - if the file is identical:
        - it will return an error and skip the upload
        - if the file is deleted on Datahub, it will undelete the file
    - if the file is different:
        - For versioned file: 
            - it will create a new version
            - if the file is deleted, it will undelete the file and create a new version
        - For an un-versioned file: it will overwrite the file

In [ ]:
try:
    glob_pattern_1: str = "*.parquet"
    glob_pattern_2: str = "*.csv"
    
    upload_response: list[CommandResponse[Contracts_DatahubCommandResponse]] = pxc.datahub.upload(local_folder=working_folder, remote_folder=datahub_working_folder, glob_patterns= [glob_pattern_1, glob_pattern_2], is_versioned=True, print_message=True)
    upload_data: Contracts_DatahubCommandResponse = SDKBase.get_response_data(upload_response)

    if upload_data is not None:
        for result in upload_data.DatahubResourceResults:
            print(f"Status: {result.Success}, Local File: {result.LocalFilePath}, Remote Destination: {result.RelativeFilePath}, Failure Reason: {result.FailureReason}")
    else:
        raise Exception("Empty response!")
            
except Exception as ex:
    print(ex)

Download (Independent of mappings and sync)

In [ ]:
try:
    glob_pattern_1: str = f"{datahub_working_folder}/**"
    download_response: list[CommandResponse[Contracts_DatahubCommandResponse]] = pxc.datahub.download(remote_glob_patterns=[glob_pattern_1], output_directory=working_folder, print_message=False)
    download_data: Contracts_DatahubCommandResponse = SDKBase.get_response_data(download_response)
    
    if download_data is not None:
        for result in download_data.DatahubResourceResults:
            print(f"Success: {result.Success}, Local File: {result.LocalFilePath}, Remote Source: {result.RelativeFilePath}, Version: {result.Version}, Failure Reason: {result.FailureReason}")
    else:
        raise Exception("Empty response!")
    
except Exception as ex:
    print(ex)


#### Download using a Manifest file (independant of mappings and sync)

In [ ]:
try:
    file_manifest: str = "./manifest_example.csv"
    download_response_with_manifest: list[CommandResponse[Contracts_DatahubCommandResponse]] = pxc.datahub.download(manifest_file_path=file_manifest, create_metadata_file=True, print_message=False)
    download_data_with_manifest: Contracts_DatahubCommandResponse = SDKBase.get_response_data(download_response_with_manifest)
    
    if download_data_with_manifest is not None:
        for result in download_data_with_manifest.DatahubResourceResults:
            print(f"Success: {result.Success}, Local File: {result.LocalFilePath}, Remote Source: {result.RelativeFilePath}, Version: {result.Version}, Failure Reason: {result.FailureReason}")    
    else:
        raise Exception("Empty response!") 

except Exception as ex:
    print(ex)


#### Search

In [ ]:
glob_pattern_1: str= f"{repo_name}/{jupyter_user}*/**"
try:
    search_response: list[CommandResponse[Contracts_DatahubSearchResponse]] = pxc.datahub.search(glob_patterns=[glob_pattern_1], print_message=True)
    search_data: Contracts_DatahubSearchResponse = SDKBase.get_response_data(search_response)
    
    if search_data is not None:
        results: list[Datahub_DatahubResourceInfo] = search_data.DatahubSearchResults #this step is necessary if your want to be able to deserialize the results
        for result in results:
            if not result.IsDeleted:
                print(f"{result.RelativePath}, Version: {result.Versions}, IsVersioned: {result.IsVersioned}")
    else:
        raise Exception("Empty response!") 
            
except Exception as ex:
    print(ex)

#### Revert the version of a mapped file to a prior version
NOTE: This will revert a locally mapped file to a prior version. It won't change anything on the server side until the files is synced again.
The local file will be overwritten!!!

In [ ]:
try:
    target_version: int = 0
    local_mapped_file_path = "./WorkingFolder/solution_data.csv"
    
    revert_response: list[CommandResponse[Contracts_DatahubCommandResponse]] = pxc.datahub.revert(file_revert_path=local_mapped_file_path, version=target_version)
    revert_data: Contracts_DatahubCommandResponse = SDKBase.get_response_data(revert_response)

    if revert_data is not None:
        print(f"{revert_data.DatahubCommandStatus}")
    else:
        raise Exception("Empty response!") 
        
except Exception as ex:
    print(ex)

#### Delete
- Un-versioned files => Hard delete
- Versioned files => Soft delete (possibility of un-delete)

Note: The deletion command will return an instant response (eventual consistency), but depending on the number of files to delete, it will get processed on the server and may take some time.

In [ ]:
try:
    glob_pattern_1: str= f"{datahub_working_folder}/*.csv"
    delete_response: list[CommandResponse[Contracts_DatahubDeleteResponse]] = pxc.datahub.delete(remote_glob_patterns=[glob_pattern_1])
    delete_data: Contracts_DatahubDeleteResponse = SDKBase.get_response_data(delete_response)
    
    if delete_data is not None:
        print(f"Success: {delete_data.Success}")
    else:
        raise Exception("Empty response!") 
        
except Exception as ex:
    print(ex)

#### Undelete (Only for Versioned files)

In [ ]:
try:
    glob_pattern_1: str= f"{datahub_working_folder}/*.csv"
    undelete_response: list[CommandResponse[Contracts_DatahubUnDeleteResponse]] = pxc.datahub.undelete(glob_patterns=[glob_pattern_1])
    undelete_data: Contracts_DatahubUnDeleteResponse = SDKBase.get_response_data(undelete_response)
    
    if undelete_data is not None:
        print(f"Success: {undelete_data.Success}")
    else:
        raise Exception("Empty response!") 
        
except Exception as ex:
    print(ex)